# QScore: Solving MaxCut with QAOA 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns

from equal1.benchmarking.algorithms import qaoa_maxcut
from networkx.algorithms.approximation.maxcut import one_exchange as guess_the_maxcut_solution


We use the QScore methodology to evaluate the performance of our QAOA implementation.

QScore is implemented by TNO [https://github.com/TNO-Quantum/qscore](https://github.com/TNO-Quantum/qscore).  Later we will try to integrate directly with the QScore project, but for now we just use the same methodology. 
We have copied over the function to compute the $\beta$ metric for MaxCut from their repository.

Graphs are Erdos-Renyi random graphs with edge probability 0.5. 
We run QAOA with 2 layers on graphs of size 4 to 10.

In [ ]:
graph = nx.erdos_renyi_graph(n=5, p=0.5,seed=87008)
nx.draw(graph, with_labels=True)


In [ ]:
guess_the_maxcut_solution(graph)

The QAI implementation is textbook, later we might use more recent approaches. 

In [ ]:
qaoa_maxcut.make_qaoa_maxcut_circuit(graph, layers=1).draw('mpl', fold=100)

## Run the Benchmark

In [ ]:
rng = np.random.default_rng()
all_results=[]
for device in ["hpc-gamma1", "hpc-conservative"]:
    results = qaoa_maxcut.compute_qscore(
        device,
        sizes=[4, 5, 6, 8, 10, 12],
        replicates=100,
        executor=lambda c: qaoa_maxcut.run_on_qbraid(c, device),
        layers=2,
        seed=int(rng.integers(0, 1000000000,1)[0]),
    )
    all_results.extend(results)
df = pd.DataFrame.from_dict(all_results)
df.to_csv(f"qaoa_maxcut.csv")


In [ ]:
results_df = pd.read_csv("qaoa_maxcut.csv", index_col=0, comment='#')
results_df

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.lineplot(
    data=results_df,
    x="graph_size",
    y="beta",
    hue="device",
    marker="o",
    ax=ax,
    errorbar='sd',
)
sns.scatterplot(
    data=df,
    x="graph_size",
    y="beta",
    hue="device",
    marker="o",
    s=20,
    ax=ax,
    legend=False,
    edgecolor="none"
)


ax.axhline(1.0, color='gray', linestyle='--')#, label='Perfect Solution')
ax.axhline(0.2, color='red', linestyle='--', label='$\\beta^*$ = 0.2')
ax.axhline(0.0, color='gray', linestyle='--')#, label='Random Sampling')
ax.legend(title="Device")
ax.set_xlabel("Graph Size (Number of Nodes)")
ax.set_ylabel("$\\beta(n)$")

In [ ]:
results_df.groupby(["device", "graph_size"]).size()
